# Урок 3. Логистическая регрессия. Log Loss.

### Задание-1:
Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.  

In [19]:
import numpy as np
import matplotlib.pyplot as plt

In [20]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [21]:
def calc_logloss(y, y_pred):
    y_pred_ = np.clip(y_pred, 1e-16, 1)
    err = - np.mean(y * np.log(y_pred_) + (1.0 - y) * np.log(1.0 - y_pred_))
    err = np.sum(err)
    return err

### Задание-2:
Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [22]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [23]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  3.        ,
         0.        ,  5.        , 10.        ,  1.        ,  2.        ],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [ 1.        ,  1.        ,  2.        ,  1.        ,  2.        ,
         1.        ,  3.        ,  3.        ,  1.        ,  2.        ]])

In [24]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [28]:
W = eval_model(X_st, y, iterations=10**3, alpha=3)

100 [-8.51673764 -1.71111938 -1.38606983  7.28158726] 0.29317933648038047
200 [-13.11264893  -2.91758836  -1.32887393  15.25013166] 1.779334266776606
300 [-16.34784194  -2.52358491  -3.58710214  14.0649044 ] 0.23209165779711988
400 [-17.9180478   -2.48236136  -4.215914    15.04778717] 0.20804906807774284
500 [-19.22367205  -2.56735548  -4.65327841  16.09142534] 0.19997872978473893
600 [-20.41984797  -2.5623619   -5.04609253  17.01262184] 0.1879798019849792
700 [-21.51452025  -2.58394493  -5.39959361  17.86189317] 0.17921032739665127
800 [-22.52683447  -2.60985149  -5.72083214  18.65211697] 0.17189744297211426
900 [-23.47006092  -2.63737709  -6.01607681  19.39263488] 0.16564286047389815
1000 [-24.35402267  -2.66399475  -6.28979022  20.09126333] 0.16018416765029295


### Задание-3:
Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [29]:
def calc_pred_proba(W, X):
    return sigmoid(np.dot(X.T, W))

In [30]:
calc_pred_proba(W, X_st)

array([0.31748792, 0.03358156, 0.99999851, 0.11279739, 0.84225541,
       0.11996466, 0.99999986, 0.01770817, 0.47084048, 0.99996194])

### Задание-4:
Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [34]:
def calc_pred(W, X, threshold=0.5):
    pred_proba = calc_pred_proba(W, X)
    pred = pred_proba >= threshold
    return pred.astype(np.int64)

In [35]:
y_pred = calc_pred(W, X_st)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 0, 1], dtype=int64)

### Задание-5:
Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [42]:
def metrics(y, y_pred):
    TP = np.sum(np.logical_and(y_pred == 1, y == 1))
    TN = np.sum(np.logical_and(y_pred == 0, y == 0))
    FP = np.sum(np.logical_and(y_pred == 1, y == 0))
    FN = np.sum(np.logical_and(y_pred == 0, y == 1))

    print(f'TP:{TP}\tTN:{TN}\tFP:{FP}\tFN:{FN}')
    
    accuracy = (TP + TN)/(TP + TN + FP + FN)
    precision = TP/(TP + FP)
    recall = TP/(TP + FN)
    f1_score = 2*precision*recall/(precision + recall)

    return accuracy, precision, recall, f1_score

metrics(y, y_pred)

TP:4	TN:5	FP:0	FN:1


(0.9, 1.0, 0.8, 0.888888888888889)

### Задание-6:
Могла ли модель переобучиться? Почему?

Могла, но у нас здесь примеры для обучения мало.

### Задание-7:
Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [46]:
# L1
def eval_model_l1(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * np.sign(W))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [50]:
L1 = eval_model_l1(X_st, y, 10**4, alpha=1e-3)
metrics(y, calc_pred(W_l1, X_st, threshold=0.5))

1000 [ 0.25805313 -0.68199349  0.68835344  1.24114036] 0.5906912246159404
2000 [ 0.14826602 -0.69312634  0.77941079  1.21302466] 0.5686262545323586
3000 [ 0.06278    -0.68610934  0.8515415   1.21294395] 0.555989255331379
4000 [-0.00868432 -0.68420411  0.90686651  1.22453939] 0.5476418424905514
5000 [-0.06998377 -0.68665665  0.94958774  1.2441288 ] 0.5416473076008509
6000 [-0.1238398  -0.69204712  0.98295251  1.26930673] 0.5369589599303932
7000 [-0.17225684 -0.69935375  1.00933208  1.29828375] 0.5330195941588649
8000 [-0.216695   -0.70786658  1.03044302  1.3297497 ] 0.5295332314112333
9000 [-0.25821004 -0.71709279  1.04752724  1.36276055] 0.5263411473515526
10000 [-0.29756284 -0.72669078  1.06148567  1.39664379] 0.523356378499251
TP:4	TN:4	FP:1	FN:1


(0.8, 0.8, 0.8, 0.8000000000000002)

In [51]:
# L2
def eval_model_l2(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * 2 * np.dot((y_pred - y), X.T) + lambda_ * W) 
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [53]:
L2 = eval_model_l2(X_st, y, 10**4, alpha=1e-3)
metrics(y, calc_pred(W_l2, X_st, threshold=0.5))

1000 [ 0.14824263 -0.6931325   0.77943781  1.2130076 ] 0.5686375217599587
2000 [-0.00870593 -0.68419854  0.90689657  1.22452819] 0.547646086619776
3000 [-0.12385726 -0.69204419  0.98297807  1.26930016] 0.5369617638002452
4000 [-0.21670787 -0.70786625  1.03046354  1.32974811] 0.5295356665030507
5000 [-0.29757211 -0.72669241  1.06150217  1.39664615] 0.5233586167104295
6000 [-0.37182646 -0.74613589  1.08249171  1.46527762] 0.517833278950102
7000 [-0.44226181 -0.76507486  1.09687272  1.53333105] 0.5127523384507873
8000 [-0.5103353  -0.78301221  1.10656178  1.5997335 ] 0.5080216036600996
9000 [-0.57681632 -0.79975967  1.1126796   1.66403917] 0.5035842166428176
10000 [-0.64211773 -0.81528008  1.11592293  1.726115  ] 0.499398897274946
TP:4	TN:4	FP:1	FN:1


(0.8, 0.8, 0.8, 0.8000000000000002)